In [1]:

pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-9ri0uxdp/gym-walk_f182a676e9ff40c99133dbd29b66cfef
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-9ri0uxdp/gym-walk_f182a676e9ff40c99133dbd29b66cfef
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=5377 sha256=09b99d9095153bfcad1074094f25bcd320297b570602a17971843ee36384f01f
  Stored in directory: /tmp/pip-ephem-wheel-cache-8ud3irzv/wheels/bf/23/e5/a94be4a90dd18f7ce958c21f192276cb01ef0daaf2bc66583b
Successfully built gym-walk


In [2]:
import warnings ; warnings.filterwarnings('ignore')

import gym
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123);


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [3]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [4]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")


def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [5]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [6]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
LEFT, DOWN, RIGHT, UP = range(4)


In [7]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [8]:
init_state

0

In [10]:
state, reward, done, info = env.step(RIGHT)
print("state:{0} - reward:{1} - done:{2} - info:{3}".format(state, reward, done, info))

state:4 - reward:0.0 - done:False - info:{'prob': 0.3333333333333333}


In [11]:
pi_frozenlake = lambda s: {
    0: RIGHT,
    1: DOWN,
    2: RIGHT,
    3: LEFT,
    4: DOWN,
    5: LEFT,
    6: RIGHT,
    7:LEFT,
    8: UP,
    9: DOWN,
    10:LEFT,
    11:DOWN,
    12:RIGHT,
    13:RIGHT,
    14:DOWN,
    15:LEFT #Stop
}[s]
print_policy(pi_frozenlake, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Policy:
| 00      > | 01      v | 02      > | 03      < |
| 04      v |           | 06      > |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [12]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_frozenlake, goal_state=goal_state) * 100,
    mean_return(env, pi_frozenlake)))

Reaches goal 10.00%. Obtains an average undiscounted return of 0.1000.


In [13]:
# Create your own policy (example: a simple one always going RIGHT unless near bottom row)
pi_2 = lambda s: {
    0: RIGHT,
    1: RIGHT,
    2: LEFT,
    3: DOWN,
    4: RIGHT,
    5: LEFT,
    6: RIGHT,
    7: DOWN,
    8: RIGHT,
    9: LEFT,
    10: RIGHT,
    11: DOWN,
    12: LEFT,
    13: DOWN,
    14: RIGHT,
    15: LEFT  # Terminal
}[s]

print("Name: NAVEEN")
print("Register Number: 212222240070")
print_policy(pi_2, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_2, goal_state=goal_state) * 100,
    mean_return(env, pi_2)))


Name: NAVEEN
Register Number: 212222240070
Policy:
| 00      > | 01      > | 02      < | 03      v |
| 04      > |           | 06      > |           |
| 08      > | 09      < | 10      > |           |
|           | 13      v | 14      > |           |
Reaches goal 2.00%. Obtains an average undiscounted return of 0.0200.


In [14]:
# Find the probability of success and the mean return of your policy

success_rate = probability_success(env, pi_2, goal_state=goal_state) * 100
avg_return = mean_return(env, pi_2)

print("Reaches goal {:.2f}% of the time.".format(success_rate))
print("Obtains an average undiscounted return of {:.4f}.".format(avg_return))

Reaches goal 2.00% of the time.
Obtains an average undiscounted return of 0.0200.


In [15]:

# Compare your policy with the first policy

# Evaluate first policy
success_frozenlake = probability_success(env, pi_frozenlake, goal_state=goal_state) * 100
avg_return_frozenlake = mean_return(env, pi_frozenlake)

In [16]:
# Evaluate your policy
success_pi2 = probability_success(env, pi_2, goal_state=goal_state) * 100
avg_return_pi2 = mean_return(env, pi_2)

print("----- Policy Comparison -----")
print("First Policy:")
print("  Success Rate: {:.2f}%".format(success_frozenlake))
print("  Mean Return : {:.4f}".format(avg_return_frozenlake))

print("\nYour Policy:")
print("  Success Rate: {:.2f}%".format(success_pi2))
print("  Mean Return : {:.4f}".format(avg_return_pi2))


def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)
    while True:
        delta = 0
        for s in range(len(P)):
            v = 0
            a = pi(s)  # action chosen by policy
            for prob, next_state, reward, done in P[s][a]:
                v += prob * (reward + gamma * V[next_state] * (not done))
            delta = max(delta, abs(v - V[s]))
            V[s] = v
        if delta < theta:
            break
    return V



----- Policy Comparison -----
First Policy:
  Success Rate: 10.00%
  Mean Return : 0.1000

Your Policy:
  Success Rate: 2.00%
  Mean Return : 0.0200


In [17]:

# Code to evaluate the first policy
V1 = policy_evaluation(pi_frozenlake, P,gamma=0.99)
print_state_value_function(V1, P, n_cols=4, prec=5)


State-value function:
| 00 0.11448 | 01 0.08191 | 02 0.13372 | 03 0.06586 |
| 04 0.15053 |           | 06 0.20562 |           |
| 08 0.30562 | 09 0.46997 | 10 0.48938 |           |
|           | 13 0.62915 | 14 0.80739 |           |


In [19]:
# Code to evaluate the second policy
V2 = policy_evaluation(pi_2, P, gamma=0.99)
print_state_value_function(V2, P, n_cols=4, prec=5)



State-value function:
| 00 0.0275 | 01 0.03162 | 02 0.06419 | 03 0.03162 |
| 04 0.02422 |           | 06 0.09871 |           |
| 08 0.04588 | 09 0.11481 | 10 0.23494 |           |
|           | 13 0.30204 | 14 0.61323 |           |


In [23]:
# Compare state by state
better_or_equal = np.all(V2 >= V1)

print("Value Function of First Policy (pi_frozenlake):")
print_state_value_function(V1, P, n_cols=4, prec=5)

print("\nValue Function of Second Policy (pi_2):")
print_state_value_function(V2, P, n_cols=4, prec=5)

print("\nPolicy Comparison Result:")
if better_or_equal:
    print("pi_2 is better than or equal to pi_frozenlake (π₂ ≥ π₁)")
else:
    print("pi_frozenlake is better than or equal to pi_2 (π₁ ≥ π₂)")



Value Function of First Policy (pi_frozenlake):
State-value function:
| 00 0.11448 | 01 0.08191 | 02 0.13372 | 03 0.06586 |
| 04 0.15053 |           | 06 0.20562 |           |
| 08 0.30562 | 09 0.46997 | 10 0.48938 |           |
|           | 13 0.62915 | 14 0.80739 |           |

Value Function of Second Policy (pi_2):
State-value function:
| 00 0.0275 | 01 0.03162 | 02 0.06419 | 03 0.03162 |
| 04 0.02422 |           | 06 0.09871 |           |
| 08 0.04588 | 09 0.11481 | 10 0.23494 |           |
|           | 13 0.30204 | 14 0.61323 |           |

Policy Comparison Result:
pi_frozenlake is better than or equal to pi_2 (π₁ ≥ π₂)


In [21]:
V1>V2


if(np.sum(V1>V2)==11):
  print("The first policy is the better policy")
elif(np.sum(V2>V1)==11):
  print("The second policy is the better policy")
else:
  print("Both policies have their merits.")

The first policy is the better policy
